# Test with 10 MI features and 30 MU points

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
 

In [9]:
token = "85yAUHFmZ0iMdi3dXtoCEM2hMIHf7nAYObWesUYpd9kBmxkXqXt1wr83ArjCC/nG"

from iqm.qiskit_iqm import IQMProvider
provider = IQMProvider("https://resonance.meetiqm.com/", quantum_computer="garnet",
            token=token)
backend = provider.get_backend()

In [ ]:
# Load dataset
df = pd.read_csv("dataset_graph7.csv")

# Separate features and labels
X = df.drop(columns=['loan_status']).values
y = df['loan_status'].values

# Split by percentage
train_percentage = 0.8  # 80% train, 20% test
X_train, X_test, train_labels, test_labels = train_test_split(
    X, y, train_size=train_percentage, random_state=42, shuffle=True
)

# Actual sizes after splitting
train_size = X_train.shape[0]
test_size = X_test.shape[0]

# Optional: print shapes to check
print("X_train shape:", X_train.shape)
print("train_labels shape:", train_labels.shape)
print("X_test shape:", X_test.shape)
print("test_labels shape:", test_labels.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_graph7.csv'

In [ ]:
# Empty kernel matrix
num_samples = np.shape(X_train)[0]
kernel_matrix = np.full((num_samples, num_samples), np.nan)
test_matrix = np.full((test_size, num_samples), np.nan)

In [ ]:
# from qiskit.circuit.library import zz_feature_map
# fm = zz_feature_map(feature_dimension=np.shape(train_data)[1], entanglement='linear', reps=1)
 
from qiskit.circuit.library import zz_feature_map, unitary_overlap
 
fm = zz_feature_map(feature_dimension=np.shape(X_train)[1],  entanglement="linear", reps=1)

NameError: name 'X_train' is not defined

### Check noise on the diagonal

In [ ]:
num_shots = 1000

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit import execute

unitary1 = fm.assign_parameters(X_train[0])
unitary2 = fm.assign_parameters(X_train[0])

overlap_circ = unitary_overlap(unitary1, unitary2)
overlap_circ.measure_all()


pm_no_opt = generate_preset_pass_manager(
    optimization_level=0,
    backend=backend
)
circ_no_opt = pm_no_opt.run(overlap_circ)


counts = (
    backend.run(circ_no_opt, shots=num_shots)
    .result()[0]
    .data.meas.get_int_counts()
)

# Probability of measuring |0...0>
p0 = counts.get('0' * circ_no_opt.num_qubits, 0) / num_shots

print("Estimated K(x,x):", p0)


### With pauli random noise and filling things

In [ ]:
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)

    sampler.options.dynamical_decoupling.enable = True
    sampler.options.twirling.enable_gates = True

    result = sampler.run(
        [circ_no_opt],
        shots=num_shots
    ).result()

    quasi = result.quasi_dists[0]
    p0 = quasi.get(0, 0.0)
    session.close()
    
print("Estimated K(x,x) with DD + twirling:", p0)

### Actual complete kernel matrix


In [ ]:
from qiskit.primitives import StatevectorSampler, unitary_overlap
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit import transpile
 
# Open a Runtime session:
session = Session(backend=backend)
# Use sampler and get the counts

sampler = Sampler(mode=session)
options = sampler.options
options.dynamical_decoupling.enable = True
options.twirling.enable_gates = True

for x1 in range(0, train_size):
    for x2 in range(x1 + 1, train_size):
        unitary1 = fm.assign_parameters(list(X_train[x1]) + [np.pi / 2])
        unitary2 = fm.assign_parameters(list(X_train[x2]) + [np.pi / 2])
 
        # Create the overlap circuit
        overlap_circ = unitary_overlap(unitary1, unitary2)
        overlap_circ.measure_all()

        print("circuit depth = ", overlap_circ.decompose().depth())
        #overlap_circ.decompose().draw("mpl", scale=0.6, style="iqp")
        print(
            "two-qubit depth",
            overlap_circ.decompose().depth(lambda instr: len(instr.qubits) > 1),
        )

        # Use sampler and get the counts

        # Transpile and run the circuit on an IQM backend
        qc_transpiled = transpile( overlap_circ, optimization_level=3, backend=backend)
        print("circuit depth = ", qc_transpiled.decompose().depth())
        print(
            "two-qubit depth",
            qc_transpiled.decompose().depth(lambda instr: len(instr.qubits) > 1),
        )
        #qc_transpiled.draw("mpl")

 
        counts = (
            sampler.run([qc_transpiled], shots=num_shots).result()[0].data.meas.get_int_counts()
        )
 
        # Assign the probability of the 0 state to the kernel matrix, and the transposed element (since this is an inner product)
        kernel_matrix[x1, x2] = counts.get(0, 0.0) / num_shots
        kernel_matrix[x2, x1] = counts.get(0, 0.0) / num_shots
    # Fill in on-diagonal elements with 1, again, since this is an inner-product corresponding to probability (or alter the code to check these entries and verify they yield 1)
    kernel_matrix[x1, x1] = 1
 
# Close session after done
session.close()
print("training done")


In [ ]:

# Similar process to above, but for testing data.
for x1 in range(0, test_size):
    for x2 in range(0, train_size):
        unitary1 = fm.assign_parameters(list(X_test[x1]) + [np.pi / 2])
        unitary2 = fm.assign_parameters(list(X_test[x2]) + [np.pi / 2])
 
        # Create the overlap circuit
        overlap_circ = unitary_overlap(unitary1, unitary2)
        overlap_circ.measure_all()
        
        qc_transpiled = transpile( overlap_circ, optimization_level=3, backend=backend)

        counts = (
            backend.run([], shots=num_shots)
            .result()[0]
            .data.meas.get_int_counts()
        )
 
        test_matrix[x1, x2] = counts.get(0, 0.0) / num_shots
 
print("test matrix done")

In [ ]:
# Save as NumPy files
np.save("kernel_matrix_quantum.npy", kernel_matrix)
np.save("test_matrix_quantum.npy", test_matrix)

In [6]:
# import a support vector classifier from a classical ML package.
from sklearn.svm import SVC
 
# Specify that you want to use a pre-computed kernel matrix
qml_svc = SVC(kernel="precomputed")

In [ ]:
# Feed in the pre-computed matrix and the labels of the training data. The classical algorithm gives you a fit.
qml_svc.fit(kernel_matrix, train_labels)

# Now use the .score to test your data, using the matrix of test data, and test labels as your inputs.
qml_score_precomputed_kernel = qml_svc.score(test_matrix, test_labels)
print(f"Precomputed kernel classification test score: {qml_score_precomputed_kernel}")

### Classical


In [7]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Classical linear SVM
linear_svc = SVC(kernel='linear', C=1.0)

# Train on your training data
linear_svc.fit(X_train, train_labels)

# Predict on test data
y_pred = linear_svc.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(test_labels, y_pred)
print("Linear SVM accuracy:", accuracy)


NameError: name 'X_train' is not defined